# VGG-16 Pruning & Quantization (No Library)

In [1]:
import sys
sys.path.append('.')

import torch
import torch.nn as nn
from collections import OrderedDict
from models.vgg import VGG

# ==================== Configuration ====================
CKPT_PATH = './checkpoints/vgg16_ckpt_best.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# ==================== Load Model ====================
model = VGG('VGG16')
ckpt = torch.load(CKPT_PATH, map_location='cpu')

# Handle DDP state_dict
state_dict = ckpt['net']
if list(state_dict.keys())[0].startswith('module.'):
    state_dict = OrderedDict([(k.replace('module.', ''), v) for k, v in state_dict.items()])
model.load_state_dict(state_dict)
model = model.to(DEVICE)
model.eval()

# ==================== Model Info ====================
print('=' * 60)
print('VGG-16 for CIFAR-10 - Model Information')
print('=' * 60)
print(f"Checkpoint: {CKPT_PATH}")
print(f"Best Accuracy: {ckpt.get('acc', 'N/A'):.2f}%" if 'acc' in ckpt else f"Accuracy: N/A")
print(f"Trained Epochs: {ckpt.get('epoch', 'N/A')}")
print(f"Device: {DEVICE}")
print()

# Total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Model Size: {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")
print()

# Layer-wise breakdown
print('Layer-wise Parameter Count:')
print('-' * 60)
conv_layers = []
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        params = sum(p.numel() for p in module.parameters())
        conv_layers.append((name, module, params))
        print(f"{name:30s} | Conv2d | in={module.in_channels:4d}, out={module.out_channels:4d}, k={module.kernel_size[0]} | {params:,} params")
    elif isinstance(module, nn.Linear):
        params = sum(p.numel() for p in module.parameters())
        print(f"{name:30s} | Linear | in={module.in_features:4d}, out={module.out_features:4d}        | {params:,} params")

print()
print('=' * 60)
print(f"Total Conv2d Layers: {len(conv_layers)}")
print(f"Total Conv2d Parameters: {sum(p for _, _, p in conv_layers):,}")
print('=' * 60)

VGG-16 for CIFAR-10 - Model Information
Checkpoint: ./checkpoints/vgg16_ckpt_best.pth
Best Accuracy: 93.07%
Trained Epochs: 86
Device: cuda

Total Parameters: 14,728,266
Trainable Parameters: 14,728,266
Model Size: 56.18 MB (FP32)

Layer-wise Parameter Count:
------------------------------------------------------------
features.0                     | Conv2d | in=   3, out=  64, k=3 | 1,792 params
features.3                     | Conv2d | in=  64, out=  64, k=3 | 36,928 params
features.7                     | Conv2d | in=  64, out= 128, k=3 | 73,856 params
features.10                    | Conv2d | in= 128, out= 128, k=3 | 147,584 params
features.14                    | Conv2d | in= 128, out= 256, k=3 | 295,168 params
features.17                    | Conv2d | in= 256, out= 256, k=3 | 590,080 params
features.20                    | Conv2d | in= 256, out= 256, k=3 | 590,080 params
features.24                    | Conv2d | in= 256, out= 512, k=3 | 1,180,160 params
features.27              